<a href="https://colab.research.google.com/github/palakagl/Marketing/blob/main/MarketBasketAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from google.colab import files
import seaborn as sns
import matplotlib.pyplot as plt

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df_train = pd.read_csv('order_products__train_detailed.csv')
print('\nNumber of rows and columns in the data set: ',df_train.shape)
print('')

#Lets look into top few rows and columns in the dataset
df_train.head(15)


Number of rows and columns in the data set:  (119352, 10)



,Unnamed: 0,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,0,1,49302,1,1,Bulgarian Yogurt,120.0,16.0,dairy eggs,yogurt
1,1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108.0,16.0,dairy eggs,other creams cheeses
2,2,1,10246,3,0,Organic Celery Hearts,83.0,4.0,produce,fresh vegetables
3,3,1,49683,4,0,Cucumber Kirby,83.0,4.0,produce,fresh vegetables
4,4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95.0,15.0,canned goods,canned meat seafood
5,5,1,13176,6,0,Bag of Organic Bananas,24.0,4.0,produce,fresh fruits
6,6,1,47209,7,0,Organic Hass Avocado,24.0,4.0,produce,fresh fruits
7,7,1,22035,8,1,Organic Whole String Cheese,21.0,16.0,dairy eggs,packaged cheese
8,8,36,39612,1,0,Grated Pecorino Romano Cheese,2.0,16.0,dairy eggs,specialty cheeses
9,9,36,19660,2,1,Spring Water,115.0,7.0,beverages,water seltzer sparkling water


In [4]:
basket_train = (df_train.groupby(['order_id', 'product_name'])['reordered']
               .sum().unstack().reset_index().fillna(0).set_index('order_id'))
basket_train

product_name,#2 Coffee Filters,#4 Natural Brown Coffee Filters,0 Calorie Fuji Apple Pear Water Beverage,0% Fat Black Cherry Greek Yogurt y,0% Fat Blueberry Greek Yogurt,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Fat Strawberry Greek Yogurt,0% Fat Superfruits Greek Yogurt,0% Greek Strained Yogurt,...,of Norwich Original English Mustard Powder Double Superfine,pumpkin spice,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Bleach Disinfectant Cleanser Scratch Free Lavender Fresh,with Crispy Almonds Cereal,with Dawn Action Pacs Fresh Scent Dishwasher Detergent Pacs,with Olive Oil Mayonnaise Dressing,with Sweet & Smoky BBQ Sauce Cheeseburger Sliders,with Xylitol Minty Sweet Twist 18 Sticks Sugar Free Gum
order_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291424,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
291481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
frequent_itemsets_train = apriori(basket_train, min_support=0.0005, 
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets_train['length'] = frequent_itemsets_train['itemsets'].apply(lambda x: len(x))

frequent_itemsets_train

,support,itemsets,length
0,0.126609,(Banana),1
1,0.103702,(Bag of Organic Bananas),1
2,0.065791,(Organic Strawberries),1
3,0.062062,(Organic Baby Spinach),1
4,0.050253,(Organic Avocado),1
...,...,...,...
6974,0.000533,"(Organic Celery Hearts, Organic Italian Parsle...",2
6975,0.000533,"(Organic Carrot Bunch, Organic Zucchini)",2
6976,0.000533,"(Asparagus, Organic Unsweetened Almond Milk)",2
6977,0.000533,(Low Fat Split Pea Soup),1


In [27]:
mba_train = association_rules(frequent_itemsets_train, metric='lift', 
                  min_threshold=0.01).sort_values('lift', ascending=False).reset_index(drop=True)
mba_train

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Dairy Free Coconut Milk Raspberry Yogurt Alte...,(Dairy Free Coconut Milk Yogurt Alternative),0.000977,0.000799,0.000533,0.545455,682.606061,0.000532,2.198242
1,(Dairy Free Coconut Milk Yogurt Alternative),(Dairy Free Coconut Milk Raspberry Yogurt Alte...,0.000799,0.000977,0.000533,0.666667,682.606061,0.000532,2.997070
2,(Organic Fiber & Protein Pear Blueberry & Spin...,"(Fiber & Protein Organic Pears, Raspberries, B...",0.000888,0.000888,0.000533,0.600000,675.780000,0.000532,2.497780
3,"(Fiber & Protein Organic Pears, Raspberries, B...",(Organic Fiber & Protein Pear Blueberry & Spin...,0.000888,0.000888,0.000533,0.600000,675.780000,0.000532,2.497780
4,(Dairy Free Coconut Milk Raspberry Yogurt Alte...,(Dairy Free Coconut Milk Blueberry Yogurt Alte...,0.000977,0.001154,0.000710,0.727273,630.097902,0.000709,3.662435
...,...,...,...,...,...,...,...,...,...
14699,(Soda),(Bag of Organic Bananas),0.010122,0.103702,0.000622,0.061404,0.592113,-0.000428,0.954934
14700,(Limes),(Strawberries),0.031253,0.036580,0.000533,0.017045,0.465978,-0.000611,0.980127
14701,(Strawberries),(Limes),0.036580,0.031253,0.000533,0.014563,0.465978,-0.000611,0.983064
14702,(Organic Hass Avocado),(Strawberries),0.044127,0.036580,0.000622,0.014085,0.385034,-0.000993,0.977183


In [28]:
mba_train.to_csv('order_products__train_detailed_lift.csv', index=False, encoding = 'utf-8-sig')
files.download('order_products__train_detailed_lift.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_prior = pd.read_csv('order_products__prior_detailed.csv')
print('\nNumber of rows and columns in the data set: ',df_prior.shape)
print('')

#Lets look into top few rows and columns in the dataset
df_prior.head(15)

In [ ]:
basket_prior = (df_prior.groupby(['order_id', 'product_name'])['reordered']
               .sum().unstack().reset_index().fillna(0).set_index('order_id'))
basket_prior

In [ ]:
frequent_itemsets_prior = apriori(basket_prior, min_support=0.0005, 
                                 use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

frequent_itemsets_prior['length'] = frequent_itemsets_prior['itemsets'].apply(lambda x: len(x))

frequent_itemsets_prior

In [ ]:
mba_prior = association_rules(frequent_itemsets_prior, metric='lift', 
                  min_threshold=0.01).sort_values('lift', ascending=False).reset_index(drop=True)
mba_prior

In [ ]:
mba_prior.to_csv('order_products__prior_detailed_lift.csv', index=False, encoding = 'utf-8-sig')
files.download('order_products__prior_detailed_lift.csv')